In [52]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env.local')

True

In [53]:
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [36]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
  name='rag', dimension=1536, metric='cosine', spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

In [37]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Roberts',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Roberts is fantastic! Her lectures are engaging and she makes complex topics easy to understand.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Calculus I',
  'stars': 3,
  'review': 'Dr. Smith is knowledgeable but his teaching style is a bit dry. The material can be hard to follow.'},
 {'professor': 'Dr. Laura Johnson',
  'subject': 'Modern History',
  'stars': 4,
  'review': 'Dr. Johnson is passionate about history and brings a lot of energy to her classes. However, her exams can be quite challenging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': "Dr. Brown's lectures are hard to follow and he doesn't provide enough examples. The subject is difficult enough without extra obstacles."},
 {'professor': 'Dr. Alice Davis',
  'subject': 'Computer Science Fundamentals',
  'stars': 5,
  'review': 'Dr. Davis is an excellent instr

In [56]:
processed_data = []
client = OpenAI(api_key="YOUR_API_KEY")

for review in data['reviews']:
  res = client.embeddings.create(
    model="text-embedding-3-small",
    input=review['review']
  )

  embedding = res.data[0].embedding
  processed_data.append({
    'values': embedding,
    "id": review['professor'],
    "metadata": {
      "review": review['review'],
      "stars": review['stars'],
      "subject": review['subject']
    }
  })

In [57]:
processed_data[0]

{'values': [-0.04245749,
  0.00020943461,
  -0.023776678,
  0.04789147,
  0.0011109469,
  0.01744911,
  0.013717779,
  0.040163144,
  -0.018318547,
  -0.024054414,
  0.020661196,
  -0.007257381,
  0.020214403,
  -0.027411407,
  -0.0040151067,
  0.0069192667,
  -0.015360048,
  -0.00758342,
  0.041877866,
  0.076848544,
  0.04736015,
  -0.027677067,
  0.07646213,
  -0.022955542,
  -0.0641451,
  -0.024175169,
  0.025938194,
  0.003915484,
  0.0065630395,
  0.03299029,
  0.057286218,
  -0.005636244,
  -0.014538913,
  -0.019103456,
  -0.014502686,
  0.04789147,
  -0.016253635,
  0.0013554759,
  0.01979176,
  0.017654395,
  0.034729164,
  -0.0031879344,
  -0.018620435,
  0.017859679,
  0.04837449,
  -0.005907943,
  -0.028522354,
  -0.0008898141,
  0.025817439,
  0.0046158633,
  -0.08399724,
  0.036468036,
  0.016326088,
  -0.034463502,
  -0.06457982,
  0.059556413,
  0.018656662,
  0.036951058,
  0.010668713,
  -0.031927645,
  0.024597812,
  -0.019550249,
  -0.011990981,
  -0.0090807825,
  -

In [59]:
index = pc.Index('rag')
index.upsert(
  vectors=processed_data,
  namespace='ns1'
)

{'upserted_count': 21}

In [60]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}